# Argumentation framework

### Random Generators 

In [1]:
import random as rd
import time
rd.seed(time.time())

#### Grounded Generator

In [2]:

def GR_gen(maxA,att_prob):
    A = rd.randint(1, maxA)
    V = list(range(1,A+1))
    unconnected = list(range(1,A+1))
    E = []
    G=(V,E)
    for i in range(A):
        for j in range(i-1):
            if (rd.random()<att_prob):
                E.append((V[i],V[j]))
                try:
                    unconnected.remove(i+1)
                except ValueError:
                    continue
    for b in unconnected:
        k = rd.randint(0, A-1)
        if (rd.randint(0, 1) == 0):
            E.append((b,V[k]))
        else:
            E.append((V[k],b))
    return G


#### Stable Generator

In [3]:
def randomSet(V,size):
    Vcop = V.copy()
    set = []
    while(len(set) < size):
        if Vcop == []:
            return set
        num = rd.randint(0,len(Vcop)-1)
        set.append(Vcop[num])
        del Vcop[num]
    return set

In [4]:
def ST_gen(maxA,minNE,maxNE,minSOE,maxSOE,minSOG,maxSOG,att_prob):
    A = rd.randint(1, maxA)
    X = rd.randint(minNE, maxNE)
    S = rd.randint(minSOE, maxSOE)
    R = rd.randint(minSOG, maxSOG)
    V = list(range(1,A+1))
    E = []
    G=(V,E)
    grounded = list(range(1,R+1))
    for i in range(1,R+1):
        for k in range (i):
            if (rd.random()<att_prob):
                E.append((V[i-1],V[k]))
    for j in range (1,X+1):
        M = randomSet(V,S)
        for i in range(R+1,A+1):
            if not V[i-1] in M:
                ak = M[rd.randint(0,len(M)-1)]
                E.append((ak,V[i-1]))
    return G

In [5]:
def tofile(G,filename):
    with open(filename,"w") as writer:
        for a in G[0]:
            writer.write(f"{a}\n")
        writer.write("#\n")
        for t in G[1]:
            writer.write(f"{t[0]} {t[1]}\n")


In [6]:
import os
from pathlib import Path
import subprocess

def solve(input: str, problem: str, arg: str = None, solvers: list[str] = None):
    exe = Path().joinpath(
        os.getcwd(),
        "..",
        "target",
        "release",
        "ter.exe",
    ).resolve()
    result = subprocess.run(
        [
            exe,
            "-p",
            problem,
            "-f",
            input,
            *([] if arg is None else ["-a", arg]),
            *([] if solvers is None else ["-s", ",".join(solvers)]),
        ],
        capture_output=True,
        text=True,
    )
    if result.returncode != 0:
        raise Exception(result.stderr)
    return result.stdout

In [7]:
print(solve("../examples/tgf.txt", "SE-CO", solvers=["dpll", "minisat"]))

[1,4,6]



In [8]:
print(solve("../examples/tgf.txt", "EE-CO"))

[[1],[4,6,1],[1,3]]



In [9]:
print(solve("../examples/tgf.txt", "DC-CO", arg="1", solvers=["manysat"]))

YES



In [10]:
print(solve("../examples/tgf.txt", "DC-CO", arg="3"))

YES



In [11]:
print(solve("../examples/tgf.txt", "DS-CO", arg="1"))

YES



In [12]:
print(solve("../examples/tgf.txt", "DS-CO", arg="3"))

NO



In [25]:
G1 = GR_gen(500,.3)

In [14]:
G2 = ST_gen(100,0,10,0,10,0,10,.3)

In [26]:
tofile(G1,"test.txt")

In [27]:
print(solve("test.txt", "SE-CO", solvers=["dpll", "minisat"]))

[479,465,463,422,462,395,324,269,474,414,464,477,480,49]



In [28]:
print(solve("test.txt", "SE-CO", solvers=["dpll"]))

[477,479,463,480,465,422,474,462,414,395,324,269,464,49]



In [29]:
print(solve("test.txt", "SE-CO", solvers=["minisat"]))

[464,479,324,480,474,465,269,463,477,49,462,395,422,414]

